In [1]:
from pydantic import BaseModel, Field, conint, root_validator, ValidationError
from ipyautoui import AutoUi

## Define a simple class with validation

In [2]:
class ApertureSettings(BaseModel):
    radius : conint(ge=1) = Field(default=1)
    inner_annulus : conint(ge=1) = Field(default=2)
    outer_annulus : conint(ge=1) = Field(default=3)

    class Config:
        validate_assignment = True
        validate_all = True

    @root_validator(skip_on_failure=True)
    def check_annuli(cls, values):
        if values['inner_annulus'] >= values['outer_annulus']:
            raise ValueError('inner_annulus must be smaller than outer_annulus')
        if values['radius'] >= values['inner_annulus']:
            raise ValueError('radius must be smaller than inner_annulus')
        return values

### Make an AutoUi and an observer

In [7]:
ui = AutoUi(ApertureSettings)

In [8]:
counter = 0
def my_observer(change):
    global counter
    counter += 1
    print(f"My observer call {counter}")
    try: 
        ApertureSettings(**ui.value)
    except ValidationError:
        print("    Bad state")
    else: 
        print("    Good state")
            
ui.observe(my_observer, names="_value")

### Set the value of the widget

In [9]:
ui.value = dict(radius=2, inner_annulus=4, outer_annulus=8)

My observer call 1
    Good state
My observer call 2
    Bad state
My observer call 3
    Bad state
My observer call 4
    Good state


In [6]:
ui.value

{'radius': 2, 'inner_annulus': 4, 'outer_annulus': 8}